# Description of the notebook

This notebook contains the code to create two different dashboards based on the [Billionaires Statistics Dataset (2023)](https://www.kaggle.com/datasets/nelgiriyewithana/billionaires-statistics-dataset). I've cleaned the original dataset to leave it without nulls and other minor inconsistencies that found throughout EDA.

The code for this uploaded process is also uploaded and can be found [here](https://www.kaggle.com/code/javiersab/billionaire-dataset-eda-statistics).

Principal characteristics:

- The first dashboard analyzes the data from a country perspective whereas the second one uses an industry-wise point of view.

- Both dashboards have two parts: one for KPIs and another for graphs.

  - The KPIs section displays some descriptive statistics such as count, total wealth accumulated, mean, median and proportions.
  - The graphs section contains different graphics to see their respective from different attributes (wealth, age, gender).
  - They are interactive and allow switch between different countries (or industries) and make comparisons between them.

Below some images of the results:

- Country dashboard

![Country Dashboard 1]()

- Industry dashboard


If you find this informative or inspirational, I'm really pleased and would appreciate if you consider leaving an upvote for others to easily discover value in this post.💎💰

## Importing libraries and data

In [1]:
# Basic libraries for data cleaning and manipulation
import pandas as pd
import numpy as np

# Data visualization packages
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import seaborn as sns
import squarify as sq
import folium as flm
from folium.plugins import MarkerCluster

# Libraries for dashboard building
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Other libraries that are going to be used throughout the project
import pprint
import textwrap
from IPython.display import display

In [2]:
# Loading the dataset
df_ready = df_ready = pd.read_csv("df_ready.csv")

## Country dashboard

In [9]:
# Initializing the Dash app

app = dash.Dash(__name__)

# Defining function to calculate metrics based on selected country
def calculate_metrics(selected_country):
    # Filtering data for the selected country
    filtered_data = df_ready[df_ready['country_of_residence'] == selected_country]
    
    # Calculating total number of billionaires in selected country
    total_billionaires = len(filtered_data)
    
    # Calculating total wealth accumulated by billionaires in selected country
    total_wealth = filtered_data['wealth'].sum()

    # Calculating mean wealth accumulated by billionaires in selected country
    mean_wealth = filtered_data['wealth'].mean()

    # Calculating median wealth accumulated by billionaires in selected country
    median_wealth = filtered_data['wealth'].median()
    
    # Calculating percentage of total number of billionaires in the country over the total presented in the dataset
    total_billionaires_percentage = (total_billionaires / len(df_ready)) * 100
    
    # Calculating percentage of total wealth in the selected country over the total presented in the dataset
    total_wealth_percentage = (total_wealth / df_ready['wealth'].sum()) * 100
    
    return total_billionaires, total_wealth, mean_wealth, median_wealth, total_billionaires_percentage, total_wealth_percentage

# Defining the layout of the app
app.layout = html.Div(style={'backgroundColor': '#fff'}, children=[
    html.H1(children='Country Dashboard'),
    
    # Dropdown menu to select country
    dcc.Dropdown(
        id='country_dropdown',
        options=[{'label': country, 'value': country} for country in df_ready.groupby('country_of_residence').size().sort_values(ascending=False).index],
        value=df_ready['country_of_residence'].unique()[0]
    ),
    
# KPI cards section
html.Div(children=[
    html.H2('KPIs section'),
    # Pair 1: Number of billionaires and Wealth Accumulated
    html.Div(children=[
        html.Div(children=[
            html.H3("Number of billionaires in the country"),
            html.H3(id="total_billionaires")
        ], className='six columns', style={'width': '48%', 'display': 'inline-block', 'background-color': '#9AE735', 'border': '1px solid black', 'padding': '5px'}),
        
        html.Div(children=[
            html.H3("Wealth accumulated (in $ billions)"),
            html.H3(id="total_wealth")
        ], className='six columns', style={'width': '48%', 'display': 'inline-block', 'background-color': '#9AE735', 'border': '1px solid black', 'padding': '5px'}),
    ], className='row'),

    # Pair 2: Mean and median wealth
    html.Div(children=[
        html.Div(children=[
            html.H3("Mean wealth (in $ billions)"),
            html.H3(id="mean_wealth")
        ], className='six columns', style={'width': '48%', 'display': 'inline-block', 'background-color': '#CCF39A', 'border': '1px solid black', 'padding': '5px'}),
        
        html.Div(children=[
            html.H3("Median wealth (in $ billions)"),
            html.H3(id="median_wealth")
        ], className='six columns', style={'width': '48%', 'display': 'inline-block', 'background-color': '#CCF39A', 'border': '1px solid black', 'padding': '5px'}),
    ], className='row'),

    # Pair 3: Proportion of billionaires and Proportion of wealth
    html.Div(children=[
        html.Div(children=[
            html.H3("Proportion of billionaires over the total"),
            html.H3(id="percentage_billionaires")
        ], className='six columns', style={'width': '48%', 'display': 'inline-block', 'background-color': '#E5F9CC', 'border': '1px solid black', 'padding': '5px'}),
        
        html.Div(children=[
            html.H3("Proportion of wealth over the total"),
            html.H3(id="percentage_wealth")
        ], className='six columns', style={'width': '48%', 'display': 'inline-block', 'background-color': '#E5F9CC', 'border': '1px solid black', 'padding': '5px'}),
    ], className='row'),
], className='row'),

    
    # Top row
    html.Div(children=[
        html.H2('Graphs section'),
        # Graph displaying billionaires by industry
        html.Div(children=[
            dcc.Graph(id='wealth_by_industry_chart'),
        ], className='six columns', style={'width': '50%', 'display': 'inline-block'}),
        
        # Graph displaying mean wealth by industry
        html.Div(children=[
            dcc.Graph(id='mean_wealth_by_industry_chart'),
        ], className='six columns', style={'width': '50%', 'display': 'inline-block'}),
    ], className='row'),

    # Bottom row
    html.Div(children=[
        # Graph displaying billionaires by age
        html.Div(children=[
            dcc.Graph(id='wealth_by_age_chart'),
        ], className='six columns', style={'width': '50%', 'display': 'inline-block'}),

        # Pie chart displaying gender distribution
        html.Div(children=[
            dcc.Graph(id='gender_distribution_chart'),
        ], className='six columns', style={'width': '50%', 'display': 'inline-block'}),
    ], className='row')
])

# Defining callbacks to update KPI cards and charts based on dropdown selection
@app.callback(
    [Output('total_billionaires', 'children'),
     Output('total_wealth', 'children'),
     Output('mean_wealth', 'children'),
     Output('median_wealth', 'children'),
     Output('percentage_billionaires', 'children'),
     Output('percentage_wealth', 'children'),
     Output('wealth_by_industry_chart', 'figure'),
     Output('wealth_by_age_chart', 'figure'),
     Output('gender_distribution_chart', 'figure'),
     Output('mean_wealth_by_industry_chart', 'figure')],
    [Input('country_dropdown', 'value')]
)
def update_dashboard(selected_country):
    # Calculating metrics
    total_billionaires, total_wealth, mean_wealth, median_wealth, total_billionaires_percentage, total_wealth_percentage = calculate_metrics(selected_country)
    
    # Filtering data for the selected country
    filtered_data = df_ready[df_ready['country_of_residence'] == selected_country]
    
    # Updating KPI cards
    total_billionaires_card = f"{total_billionaires:,}"
    total_wealth_card = f"${total_wealth:,.2f}"
    mean_wealth_card = f'${mean_wealth:,.2f}'
    median_wealth_card = f'${median_wealth:,.2f}'
    total_billionaires_percentage_card = f"{total_billionaires_percentage:.2f}%"
    total_wealth_percentage_card = f"{total_wealth_percentage:.2f}%"
    
    # Updating charts

    # Chart 1: Wealth by Industry
    industry_counts = filtered_data['industry'].value_counts().reset_index()
    industry_counts.columns = ['Industry', 'Billionaire count']
    industry_counts = industry_counts.sort_values(by='Billionaire count', ascending=True)
    industry_counts 
    fig_wealth_by_industry = px.bar(industry_counts,
                                    x='Billionaire count',
                                    y='Industry',
                                    orientation = 'h',
                                    title=f'Billionaires by Industry in {selected_country}', color_discrete_sequence=px.colors.qualitative.Pastel)
    
    fig_wealth_by_industry.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                                         xaxis=dict(showgrid=True,
                                                    gridcolor='black',
                                                    gridwidth=0.25,
                                                    griddash='dash'),
                                        yaxis=dict(tickfont=dict(size=10)))
    
    fig_wealth_by_industry.update_traces(marker_line_color='black',
                                         marker_line_width=1)
    
    # Chart 2: Mean Wealth by Industry
    mean_wealth_by_industry = filtered_data.groupby('industry')['wealth'].mean().reset_index().sort_values(by='wealth', ascending=True)
    mean_wealth_by_industry.columns = ['Industry', 'Mean Wealth']
    fig_mean_wealth_by_industry = px.bar(mean_wealth_by_industry,
                                         x='Mean Wealth',
                                         y='Industry',
                                         color='Mean Wealth',
                                         orientation = 'h',
                                         title=f'Billionaires\' Mean Wealth by Industry in {selected_country}')
    # Specifying hovertemplate format
    fig_mean_wealth_by_industry.update_traces(hovertemplate= "Industry Mean Wealth: $%{x:,.2f} B")
    fig_mean_wealth_by_industry.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                                              xaxis=dict(showgrid=True,
                                                         gridcolor='black',
                                                         gridwidth=0.25,
                                                         griddash='dash'),
                                              yaxis=dict(tickfont=dict(size=10)))
    

    # Chart 3: Wealth by Age
    fig_wealth_by_age = px.histogram(filtered_data, 
                                     x='age',
                                     title=f'Billionaires\' Ages distribution in {selected_country}', color_discrete_sequence=px.colors.qualitative.Pastel,
                                     nbins=round((filtered_data['age'].max() - filtered_data['age'].min()) / 5),
                                     opacity=0.75)
    

    fig_wealth_by_age.update_traces(marker_line_color='black', marker_line_width=1)
    fig_wealth_by_age.update_yaxes(tickformat=',.0f')
    fig_wealth_by_age.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                                    yaxis=dict(showgrid=True,
                                                         gridcolor='black',
                                                         gridwidth=0.25,
                                                         griddash='dash'),
                                    yaxis_range=[0,len(filtered_data) / 2.75], xaxis_range=[20, 100])

    # Chart 4: Gender Distribution
    gender_counts = filtered_data['gender'].value_counts().reset_index()
    gender_counts['gender'] = gender_counts['gender'].map({'M': 'Male', 'F': 'Female'})
    gender_counts.columns = ['Gender', 'Count']
    fig_gender_distribution = px.pie(gender_counts,
                                     names='Gender',
                                     values='Count',
                                     title=f'Billionaires\' Gender Distribution in {selected_country}',
                                     color_discrete_sequence=px.colors.qualitative.Set3)
    fig_gender_distribution.update_traces(marker=dict(line=dict(color='black', width=1)), textinfo='percent',texttemplate='%{percent}')
    
    
    return total_billionaires_card, total_wealth_card, mean_wealth_card, median_wealth_card, total_billionaires_percentage_card, total_wealth_percentage_card, fig_wealth_by_industry, fig_wealth_by_age, fig_gender_distribution, fig_mean_wealth_by_industry

# Running the Dash app
if __name__ == '__main__':
    app.run_server(mode='inline', port=8050, debug=True)



## Industry dashboard

In [8]:
# Initializing the Dash app

app = dash.Dash(__name__)

# Defining function to calculate metrics based on selected industry
def industry_metrics(industry):

    # Filtering the dataframe for the selected industry
    industry_filter = df_ready[df_ready['industry'] == industry]

    # Calculating industry leaders & followers
    # By size (country name)
    ind_size_leader = industry_filter.groupby('country_of_residence')['full_name'].count().idxmax()
    ind_size_follow = industry_filter.groupby('country_of_residence')['full_name'].count().sort_values(ascending=False).index[1]

    # By size (counts)
    ind_size_leader_c = industry_filter.groupby('country_of_residence')['full_name'].count().max()
    ind_size_follow_c = industry_filter.groupby('country_of_residence')['full_name'].count().sort_values(ascending=False).iloc[1]

    # f-strings for size
    ind_size_leader_str = f'The country with the biggest billionaire group in the {industry} industry is {ind_size_leader} with {ind_size_leader_c} billionaires.'
    ind_size_follow_str = f'The country with the second biggest billionaire group in the {industry} industry is {ind_size_follow} with {ind_size_follow_c} billionaires.'

    # By wealth (country name)
    ind_wealth_leader = industry_filter.groupby('country_of_residence')['wealth'].sum().idxmax()
    ind_wealth_follow = industry_filter.groupby('country_of_residence')['wealth'].sum().sort_values(ascending=False).index[1]

    # By wealth (wealth accumulated)
    ind_wealth_leader_s = industry_filter.groupby('country_of_residence')['wealth'].sum().sort_values(ascending=False).iloc[0]
    ind_wealth_follow_s = industry_filter.groupby('country_of_residence')['wealth'].sum().sort_values(ascending=False).iloc[1]

    # f-strings for wealth
    ind_wealth_leader_str = f'The country with the highest accumulated wealth in the {industry} is {ind_wealth_leader} with ${ind_wealth_leader_s:,.0f} billions.'
    ind_wealth_follow_str = f'The country with the second highest accumulated wealth in the {industry} industry is {ind_wealth_follow} with ${ind_wealth_follow_s:,.0f} billions.'

    # Calculating KPIs
    total_bill = len(industry_filter) # Number of billionaires in the industry
    ind_total_wealth = industry_filter['wealth'].sum() # Industry total wealth
    ind_mean_wealth = industry_filter['wealth'].mean() # Industry mean wealth
    ind_median_wealth = industry_filter['wealth'].median() # Industry median wealth
    p_total_bill = (total_bill / len(df_ready['industry'])*100) # Prop of total billionaires in industry over total
    p_total_wealth = ind_total_wealth / df_ready['wealth'].sum() * 100 # Prop of total wealth in industry over total

    return ind_size_leader_str, ind_size_follow_str, ind_wealth_leader_str, ind_wealth_follow_str, total_bill, ind_total_wealth, ind_mean_wealth, ind_median_wealth, p_total_bill, p_total_wealth

# Defining the layout of the app
app.layout = html.Div(style={'font-family': 'Arial, sans-serif', 'backgroundColor': '#fff'}, children=[
    html.H1(children='Industry Dashboard', style={'margin': '20px'}),
    
    # Dropdown menu to select country
    dcc.Dropdown(
        id='industry_dropdown',
        options=[{'label': industry, 'value': industry} for industry in df_ready.groupby('industry').size().sort_values(ascending=True).index],
        value=df_ready['industry'].unique()[0]
    ),
    
    # Section 1. Industry leaders & followers
    html.Div(children=[
        html.H2('Leaders & followers'),
        
        # Pair 1: Leaders and followers by size
        html.Div(children=[
            html.Div(children=[
                html.H3('⭐Industry leader by size'),
                html.H5(id='ind_size_leader_str'),
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#84D7F7',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-left-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right': '20px',
                                               'margin-bottom': '20px'}),
            html.Div(children=[
                html.H3("📈Industry follower by size"),
                html.H5(id="ind_size_follow_str")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#C1EBFB',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-right-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
        ], className='row', style={'display': 'flex', 'justify-content': 'space-between'}),
        
        # Pair 2: Leaders and followers by wealth
        html.Div(children=[
            html.Div(children=[
                html.H3("⭐Industry leader by wealth"),
                html.H5(id="ind_wealth_leader_str")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#84D7F7',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-bottom-left-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right': '20px',
                                               'margin-bottom': '20px'}),
            html.Div(children=[
                html.H3("📈Industry follower by wealth"),
                html.H5(id="ind_wealth_follow_str")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#C1EBFB', 
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-bottom-right-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
        ], className='row',style={'display': 'flex', 'justify-content': 'space-between'}),
    ], className='row', style={'margin': '20px'}),

    # Section 2. Industry highlights
    html.Div(children=[
        html.H2('Highlights'),
        
        # Pair 1: Number of billionaires and Wealth accumulated
        html.Div(children=[
            html.Div(children=[
                html.H3("⚡Number of billionaires"),
                html.H3(id="total_bill")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#27DDC2',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-left-radius': '25px',
                                               'border-bottom-left-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
            
            html.Div(children=[
                html.H3("💰Wealth accumulated (in $ billions)"),
                html.H3(id="ind_total_wealth")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#27DDC2',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-right-radius': '25px',
                                               'border-bottom-right-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
        ], className='row', style={'display': 'flex', 'justify-content': 'space-between'}),

        # Pair 2: Mean and median wealth
        html.Div(children=[
            html.Div(children=[
                html.H3("💰Mean wealth (in $ billions)"),
                html.H3(id="ind_mean_wealth")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#93EEE0',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-left-radius': '25px',
                                               'border-bottom-left-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
            
            html.Div(children=[
                html.H3("💰Median wealth (in $ billions)"),
                html.H3(id="ind_median_wealth")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#93EEE0',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-right-radius': '25px',
                                               'border-bottom-right-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
        ], className='row',style={'display': 'flex', 'justify-content': 'space-between'}),

        # Pair 3: Proportion of billionaires and Proportion of wealth
        html.Div(children=[
            html.Div(children=[
                html.H3("💎Proportion of billionaires over total"),
                html.H3(id="p_total_bill")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#C9F6EF',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-left-radius': '25px',
                                               'border-bottom-left-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
            
            html.Div(children=[
                html.H3("💎Proportion of wealth over total"),
                html.H3(id="p_total_wealth")
            ], className='six columns', style={'width': '40%',
                                               'display': 'inline-block',
                                               'background-color': '#C9F6EF',
                                               'border': '1px solid black',
                                               'padding': '5px',
                                               'border-top-right-radius': '25px',
                                               'border-bottom-right-radius': '25px',
                                               'margin-left': '20px',
                                               'margin-right':'20px',
                                               'margin-bottom': '20px'}),
        ], className='row',style={'display': 'flex', 'justify-content': 'space-between'}),
    ], className='row', style={'margin': '20px'}),

    # Section 3. Graphs section
    html.Div(children=[
        html.H2('Graphs section'),
        
        # Top row
        html.Div(children=[
            # Graph displaying billionaires by industry
            html.Div(children=[
                dcc.Graph(id='group_size_chart'),
            ], className='six columns', style={'width': '50%',
                                               'display': 'inline-block',
                                               'height': '400px'}),
            
            # Graph displaying mean wealth by industry
            html.Div(children=[
                dcc.Graph(id='group_wealth_chart'),
            ], className='six columns', style={'width': '50%',
                                               'display': 'inline-block',
                                               'height': '400px'}),
        ], className='row'),

        # Middle row
        html.Div(children=[
            # Histogram displaying billionaires by age
            html.Div(children=[
                dcc.Graph(id='age_dist_chart'),
            ], className='six columns', style={'width': '50%',
                                               'display': 'inline-block',
                                               'height': '450px'}),
            
            # Pie chart displaying gender distribution
            html.Div(children=[
                dcc.Graph(id='gender_dist_chart'),
            ], className='six columns', style={'width': '50%',
                                               'display': 'inline-block',
                                               'height': '450px'}),
        ], className='row'),

        # Bottom row
        html.Div(children=[
            # Bar chart displaying Top 10 wealthiest individuals by industry
            html.Div(children=[
                dcc.Graph(id='top_10_chart'),
            ], className='six columns', style={'width':'100%',
                                               'display': 'inline-block',
                                               'height': '550px'}),
        ], className='row'),
    ], style={'margin': '20px'}),
])


# Defining callbacks to update KPI cards and charts based on dropdown selection
@app.callback(
    [Output('ind_size_leader_str', 'children'),
     Output('ind_size_follow_str', 'children'),
     Output('ind_wealth_leader_str', 'children'),
     Output('ind_wealth_follow_str', 'children'),
     Output('total_bill', 'children'),
     Output('ind_total_wealth', 'children'),
     Output('ind_mean_wealth', 'children'),
     Output('ind_median_wealth', 'children'),
     Output('p_total_bill', 'children'),
     Output('p_total_wealth', 'children'),
     Output('group_size_chart', 'figure'),
     Output('group_wealth_chart', 'figure'),
     Output('age_dist_chart', 'figure'),
     Output('gender_dist_chart', 'figure'),
     Output('top_10_chart', 'figure')],
    [Input('industry_dropdown', 'value')]
)
def update_industry_dashboard(industry):
    # Calculating metrics
    ind_size_leader_str, ind_size_follow_str, ind_wealth_leader_str, ind_wealth_follow_str, total_bill, ind_total_wealth, ind_mean_wealth, ind_median_wealth, p_total_bill, p_total_wealth = industry_metrics(industry)
    
    # Filtering data for the selected country
    industry_filter = df_ready[df_ready['industry'] == industry]
    
    # Updating KPI cards
    ind_size_leader_str_card = f"{ind_size_leader_str}"
    ind_size_follow_str_card = f"{ind_size_follow_str}"
    ind_wealth_leader_str_card = f"{ind_wealth_leader_str}"
    ind_wealth_follow_str_card = f"{ind_wealth_follow_str}"
    total_bill_card = f"{total_bill:,}"
    ind_total_wealth_card = f"${ind_total_wealth:,.2f}"
    ind_mean_wealth_card = f'${ind_mean_wealth:,.2f}'
    ind_median_wealth_card = f'${ind_median_wealth:,.2f}'
    p_total_bill_card = f"{p_total_bill:.2f}%"
    p_total_wealth_card = f"{p_total_wealth:.2f}%"
    
    # Updating charts

    # Chart 1. Billionaires' group size by industry

    size_by_country = industry_filter.groupby('country_of_residence')['wealth'].count().reset_index()
    size_by_country_sorted = size_by_country.sort_values(by='wealth', ascending=False)

    # Separating top 3 countries and the rest
    top_countries_size = size_by_country_sorted.head(3)
    others_total_size = size_by_country_sorted.iloc[3:]['wealth'].sum()

    # Creating a DataFrame for top 3 countries and "Others"
    df_ind_size = pd.concat([top_countries_size, pd.DataFrame({'country_of_residence': ['Other countries'], 'wealth': [others_total_size]})])
    top_country_size = size_by_country_sorted.head(1)
    
    fig_group_size = px.pie(df_ind_size,
                            values='wealth',
                            names='country_of_residence',
                            title= f'Top 3 countries with most billionaires in the <br>{industry} industry', 
                            color_discrete_sequence=px.colors.qualitative.T10)

    # Updating layout to pull out the "Others" slice
    pull_value_size = 0.1
    fig_group_size.update_traces(pull=[pull_value_size if country == top_country_size['country_of_residence'].iloc[0] else 0 for country in size_by_country_sorted['country_of_residence']],
                                 sort=False)

    # Updating hover label format to ensure consistency
    fig_group_size.update_traces(marker=dict(line=dict(color='black', width=1)),
                                 hovertemplate='%{label}: %{value:,.0f}')

    fig_group_size.update_layout(
    showlegend=True,
    legend=dict(bordercolor='black', borderwidth=1,
                traceorder='normal'),
                annotations=[
                            dict(
                                text = f'The country with the biggest group of billionaires in the <br><b>{industry}</b> industry is <span style= "color: rgb(76,120,168)"><b>{top_country_size["country_of_residence"].iloc[0]}</b></span> with<span style= "color: rgb(76,120,168)"><b> {top_country_size["wealth"].iloc[0]:,.0f}</b></span>.',
                                x=-0.1,
                                y=-0.1,
                                showarrow=False,
                                xref='paper',
                                yref='paper',
                                align='left',
                                font=dict(
                                            size=12,
                                            color='black',
                                            family= 'Arial, sans-serif'
                                           )
                                       )
                            ]
    )
    
    # Chart 2. Billionaires' group wealth by industry

    wealth_by_country = industry_filter.groupby('country_of_residence')['wealth'].sum().reset_index()
    wealth_by_country_sorted = wealth_by_country.sort_values(by='wealth', ascending=False)

    # Separating top 3 countries and the rest
    top_countries_wealth = wealth_by_country_sorted.head(3)
    others_total_wealth = wealth_by_country_sorted.iloc[3:]['wealth'].sum()

    # Creating a DataFrame for top 3 countries and "Others"
    df_ind_wealth = pd.concat([top_countries_wealth, pd.DataFrame({'country_of_residence': ['Other countries'], 'wealth': [others_total_wealth]})])
    top_country_wealth = wealth_by_country_sorted.head(1)
    
    fig_group_wealth = px.pie(df_ind_wealth,
                            values='wealth',
                            names='country_of_residence',
                            title= f'Top 3 countries with most wealth in the <br>{industry} industry', 
                            color_discrete_sequence=px.colors.qualitative.T10)

    # Updating layout to pull out the "Others" slice
    pull_value_wealth = 0.1
    fig_group_wealth.update_traces(pull=[pull_value_wealth if country == top_country_wealth['country_of_residence'].iloc[0] else 0 for country in wealth_by_country_sorted['country_of_residence']],
                                 sort=False)

    # Updating hover label format to ensure consistency
    fig_group_wealth.update_traces(marker=dict(line=dict(color='black', width=1)), hovertemplate='%{label}: $%{value:,.2f}')

    fig_group_wealth.update_layout(showlegend=True,
                                   legend=dict(bordercolor='black', borderwidth=1, traceorder='normal'),
                                   annotations=[
                                       dict(
                                           text = f'The country with the greatest accumulation of wealth in the <br><b>{industry}</b> industry is <span style= "color: rgb(76,120,168)"><b>{top_country_wealth["country_of_residence"].iloc[0]}</b></span> with<span style= "color: rgb(76,120,168)"><b> ${top_country_wealth["wealth"].iloc[0]:,.0f}</b></span> billions.',
                                           x=-0.1,
                                           y=-0.1,
                                           showarrow=False,
                                           xref='paper',
                                           yref='paper',
                                            align='left',
                                           font=dict(
                                               size=12,
                                               color='black',
                                               family= 'Arial, sans-serif'
                                           )
                                       )
                                   ]
    )

    # Chart 3. Age distribution

    # Defining bin parameters
    num_bins = 9
    bin_edges = list(range(15,106,10)) # Create bins every 10 years from 15 to 105 

    fig_age_dist = px.histogram(industry_filter,
                                x='age',
                                title=f'Age Distribution in the <br>{industry} industry',
                                labels={'age': 'Age', 'count': 'Number of Individuals'},
                                nbins=len(bin_edges)-1,
                                range_x = [15,105],
                                color_discrete_sequence=px.colors.qualitative.T10)

    # Calculating histogram data
    hist_data, bin_edges = np.histogram(industry_filter['age'], bins=num_bins, range=(20,110))

    # Find the indices of the bins with the highest counts
    max_count_indices = np.where(hist_data == np.max(hist_data))[0]

    # Get the ranges of the bins with the highest counts
    bin_ranges = [int(bin_edges[i]) for i in max_count_indices]

    # Get the count of values corresponding to the highest bin
    age_max_count = np.max(hist_data)

    # Generate the annotation text
    if len(bin_ranges) == 1:
        hist_text = f"The <b>most common age range</b> in the <br><b>{industry}</b> industry is {bin_ranges[0]}-{bin_ranges[0] + 9} years old with <b>{age_max_count}</b> billionaires."
    else:
        hist_text = f"The <b>most common age ranges</b> in the <br><b>{industry}</b> industry are "
        for i, bin_range in enumerate(bin_ranges):
            if i > 0:
                hist_text += " and "
            hist_text += f"{bin_range}-{bin_range + 9}"
        hist_text += f' years old with <b>{age_max_count}</b> billionaires.'

    fig_age_dist.update_traces(marker_line_color='black', marker_line_width=1)
    
    # Customizing the layout
    fig_age_dist.update_layout(xaxis_title='Age',
                  yaxis_title='Number of Individuals',
                  plot_bgcolor='rgba(0,0,0,0)',
                  yaxis=dict(showgrid=True,
                             gridcolor='black',
                             gridwidth=0.25,
                             griddash='dash'),
                             annotations=
                               [dict(text=hist_text,
                                     x= -0.1,
                                     y= -0.30,
                                     xref='paper',
                                     yref='paper',
                                     showarrow=False,
                                     align='left',
                                     font=dict(
                                        size=12,
                                        color='black',
                                        family= 'Arial, sans-serif'
                                     ))]
    )

    # Chart 4. Gender distribution

    # Calculating gender counts for the industry
    gender_counts = industry_filter['gender'].value_counts().reset_index()
    gender_counts['gender'] = gender_counts['gender'].map({'M': 'Male', 'F': 'Female'})
    gender_counts.columns = ['Gender', 'Count']

    # Calculating overall gender distribution
    ov_gender_counts = df_ready['gender'].value_counts(normalize=True).reset_index()
    ov_gender_counts['gender'] = ov_gender_counts['gender'].map({'M': 'Male', 'F': 'Female'})
    ov_gender_counts.columns = ['Gender', 'Overall percentage']

    # Building f-string for industry percentages
    ind_gender_dist_str = '\n'.join([f"The gender distribution in the {industry} industry is: <br>Male: <b><span style= 'color: rgb(76,120,168)'>{gender_counts.loc[gender_counts['Gender'] == 'Male', 'Count'].values[0] / gender_counts['Count'].sum():.2%}</b></span>, Female: <span style= 'color: rgb(245,133,24)'><b>{gender_counts.loc[gender_counts['Gender'] == 'Female', 'Count'].values[0] / gender_counts['Count'].sum():.2%}</b></span>." for industry in industry_filter['industry'].unique()])

    # Building f-string for overall percentages
    ov_gender_dist_str = f"The overall gender distribution is: <br>Male: <b><span style='color: rgb(76,120,168)'>{ov_gender_counts.loc[ov_gender_counts['Gender'] == 'Male', 'Overall percentage'].values[0]:.2%}</b></span>, </span> Female: <b><span style= 'color: rgb(245,133,24)'>{ov_gender_counts.loc[ov_gender_counts['Gender'] == 'Female', 'Overall percentage'].values[0]:.2%}.</b></span><br>"

    # Building the f-string for comparison
    g_comp_str = '\n'.join([
    f"<br>The {industry} industry is {'<b>less</b>' if gender_counts.loc[gender_counts['Gender'] == 'Male', 'Count'].values[0] / gender_counts['Count'].sum() > ov_gender_counts.loc[ov_gender_counts['Gender'] == 'Male', 'Overall percentage'].values[0] else '<b>more</b>'} <b>equal than overall</b>."
    for industry in industry_filter['industry'].unique()
])

    fig_gender_dist = px.pie(gender_counts,
                                 names='Gender',
                                 values='Count',
                                 title=f'Gender Distribution in the <br>{industry} industry',
                                 color_discrete_sequence=px.colors.qualitative.T10)

    # Updating traces
    fig_gender_dist.update_traces(marker=dict(line=dict(color='black', width=1)),
                                      textinfo='percent',
                                      texttemplate='%{percent}')

    fig_gender_dist.update_layout(showlegend=True,
                                  legend=dict(bordercolor='black', borderwidth=1, traceorder='normal'),
                                  annotations = [ 
                                  dict(
                                      text = f'{ind_gender_dist_str}<br>{ov_gender_dist_str}</b>{g_comp_str}',
                                      x= -0.1,
                                      y= -0.4,
                                      showarrow = False,
                                      xref='paper',
                                      yref='paper',
                                      align='left',
                                      font=dict(
                                          size=12,
                                          color='black',
                                          family= 'Arial, sans-serif'
                                          )
                                        )
                                    ],
                                    margin=dict(b=100)
                                )
    
    # Chart 5. Top 10 wealthiest individuals

    # Sorting individuals by wealth in descending order
    wealthiest_ind = industry_filter.sort_values(by='wealth', ascending=False)

    # Extracting the top N wealthiest individuals
    wealthiest_ind = wealthiest_ind.head(10)

    # Metrics for f-string
    top_1_ind = wealthiest_ind.iloc[0]['wealth']
    top_10_ind = wealthiest_ind.iloc[-1]['wealth']
    top_10_p = round((wealthiest_ind['wealth'].sum() / industry_filter['wealth'].sum())*100 ,2)

    # f-strings creation
    top_10_str_1 = f"The first individual in the Top 10 of the {industry} industry has a wealth of <b><span style= 'color: rgb(8,48,107)'>${top_1_ind:,.2f}.</b></span><br>The last individual in the Top 10 has a wealth of ${top_10_ind:,.2f}. <br>Top 1 is <b><span style= 'color: rgb(8,48,107)'>{round(top_1_ind / top_10_ind,1)}x </b></span>times wealthier than individual in position 10."

    top_10_str_2 = f"These individuals concentrate the <b>{top_10_p}%</b> of {industry} industry\'s total wealth"

    # Creating a horizontal bar chart using Plotly Express
    fig_top_10 = px.bar(wealthiest_ind, 
             x='wealth',
             y='full_name',
             orientation='h',
             title= f'Top 10 Wealthiest Individuals in the {industry} industry',
             labels={'wealth': 'Wealth (in $ Billions)',
                     'full_name': 'Name',
                     'country_of_residence': 'Country'},
             color='wealth',
             color_continuous_scale='Blues',
             hover_data={'country_of_residence': True, 'wealth': True}
             )

    # Customizing the layout
    fig_top_10.update_layout(xaxis_title='Wealth (in billion $)',
                  yaxis_title='Full name',
                  yaxis_categoryorder='total ascending',  # Order bars by total value
                  xaxis_tickprefix='$',  # Add dollar sign to x-axis ticks
                  plot_bgcolor='rgba(0,0,0,0)',
                  margin=dict(b=150, l=200),
                  annotations = [
                      dict(
                          text= f"{top_10_str_1}<br>{top_10_str_2}",
                          x= -0.20,
                          y=-0.6,
                          showarrow=False,
                          xref='paper',
                          yref='paper',
                          align='left',
                          font=dict(
                            size=12,
                            color='black',
                            family= 'Arial, sans-serif',
                          )
                      )],
                  xaxis=dict(showgrid=True,
                             gridcolor='black',
                             gridwidth=0.25,
                             griddash='dash')
                             )
    

    fig_top_10.update_traces(marker_line_color='black',
                             marker_line_width=1)

    return ind_size_leader_str_card, ind_size_follow_str_card, ind_wealth_leader_str_card, ind_wealth_follow_str_card, total_bill_card, ind_total_wealth_card, ind_mean_wealth_card, ind_median_wealth_card, p_total_bill_card, p_total_wealth_card, fig_group_size, fig_group_wealth, fig_age_dist, fig_gender_dist, fig_top_10

# Running the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
